# CNN Architecture – Malaria Detection

This notebook defines a convolutional neural network for binary
classification of malaria-infected blood cells.
and we build it from scratch

*The libraries necessary*

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets,transforms,models
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
import os
from pathlib import Path
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image
import warnings
import json
import time
from datetime import datetime
from tqdm.notebook import tqdm
from sklearn.metrics import (
    classification_report, 
    confusion_matrix,
    roc_curve, 
    auc, 
    roc_auc_score,
    precision_recall_curve,
    f1_score,
    accuracy_score
)
batch_size = 32
sns.set_palette("husl")
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
# Cell 8: Define CNN Architecture from Scratch
"""
Build a Convolutional Neural Network from scratch
Architecture: Multiple conv blocks + fully connected layers
"""

class MalariaCNN(nn.Module):
    """
    Custom CNN for Malaria Detection
    
    Architecture:
    - 4 Convolutional blocks (Conv -> BatchNorm -> ReLU -> MaxPool)
    - Fully connected layers with dropout
    - Output: 2 classes (Parasitized, Uninfected)
    """
    
    def __init__(self, num_classes=2, dropout_rate=0.5):
        super(MalariaCNN, self).__init__()
        
        # ==========================
        # CONVOLUTIONAL LAYERS
        # ==========================
        
        # Block 1: 3 -> 32 channels
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, 
                              kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(32)
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        
        # Block 2: 32 -> 64 channels
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, 
                              kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(64)
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)
        
        # Block 3: 64 -> 128 channels
        self.conv3 = nn.Conv2d(in_channels=64, out_channels=128, 
                              kernel_size=3, padding=1)
        self.bn3 = nn.BatchNorm2d(128)
        self.pool3 = nn.MaxPool2d(kernel_size=2, stride=2)
        
        # Block 4: 128 -> 256 channels
        self.conv4 = nn.Conv2d(in_channels=128, out_channels=256, 
                              kernel_size=3, padding=1)
        self.bn4 = nn.BatchNorm2d(256)
        self.pool4 = nn.MaxPool2d(kernel_size=2, stride=2)
        
        
        self.flatten_size = 10*10*256
        
        # ==========================
        # FULLY CONNECTED LAYERS
        # ==========================
        
        self.fc1 = nn.Linear(self.flatten_size, 512)
        self.dropout1 = nn.Dropout(dropout_rate)
        
        self.fc2 = nn.Linear(512, 256)
        self.dropout2 = nn.Dropout(dropout_rate)
        
        self.fc3 = nn.Linear(256, num_classes)
        
    def forward(self, x):
        """
        Forward pass
        
        Args:
            x: Input tensor [batch_size, 3, 160, 160]
        
        Returns:
            Output tensor [batch_size, num_classes]
        """
        
        # Block 1
        x = self.conv1(x)
        x = self.bn1(x)
        x = F.relu(x)
        x = self.pool1(x)
        
        # Block 2
        x = self.conv2(x)
        x = self.bn2(x)
        x = F.relu(x)
        x = self.pool2(x)
        
        # Block 3
        x = self.conv3(x)
        x = self.bn3(x)
        x = F.relu(x)
        x = self.pool3(x)
        
        # Block 4
        x = self.conv4(x)
        x = self.bn4(x)
        x = F.relu(x)
        x = self.pool4(x)
        
        # Flatten
        x = x.view(x.size(0), -1)
        
        # FC layers
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout1(x)
        
        x = self.fc2(x)
        x = F.relu(x)
        x = self.dropout2(x)
        
        x = self.fc3(x)
        
        return x
# Initialize model
model = MalariaCNN(num_classes=2, dropout_rate=0.5)
model = model.to(device)

# Count parameters
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

print("\n" + "="*70)
print("MODEL INITIALIZATION")
print("="*70)
print(f"\n Model created and moved to {device}")
print(f"\n   Total parameters:     {total_params:,}")
print(f"   Trainable parameters: {trainable_params:,}")
print(f"   Model size:           ~{total_params * 4 / 1e6:.2f} MB")
print("="*70)



CNN ARCHITECTURE DEFINED

 MalariaCNN class created!

Architecture summary will be displayed in the next cell.


=================================================================
COMPLETE TRAINING, EVALUATION & DEPLOYMENT
=================================================================
Purpose: Train the model, evaluate performance, and prepare for deployment

This notebook covers:
1. Training loop with validation
2. Learning rate scheduling
3. Model checkpointing
4. Performance evaluation
5. Confusion matrix and metrics
6. ROC curve and AUC
7. Visualization of predictions
8. Model saving for deployment
9. Inference on new images

=================================================================

**final data preparations and loading for the training and testing and evaluating**

In [ ]:
train_healthy = Path('../data/processed/train/uninfected')
train_infected = Path('../data/processed/train/infected')
test_healthy = Path('../data/processed/test/uninfected')
test_infected = Path('../data/processed/test/infected')
val_healthy = Path('../data/processed/val/uninfected')
val_infected = Path('../data/processed/val/infected')
raw_base_dir = Path('../data/raw/cell_images')
raw_parasited_dir = raw_base_dir / 'Parasitized'
raw_uninfected_dir = raw_base_dir / 'Uninfected'
healthy_and_infected = Path("../data/processed")
train_dir = healthy_and_infected /"train"
val_dir = healthy_and_infected /"val"
test_dir = healthy_and_infected / "test"
train_transform = transforms.Compose([
    transforms.Resize((160,160)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomRotation(degrees=10),
    transforms.ColorJitter(
        brightness = 0.2,
        contrast = 0.2,
        saturation = 0.2,
        hue = 0.1
    ),
    transforms.ToTensor(),
    transforms.Normalize(mean = [0.485, 0.456, 0.406],std = [0.229, 0.224, 0.225])
])
eval_transforms = transforms.Compose([
    transforms.Resize((160,160)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225])
])
full_train_dataset = ImageFolder(root =train_dir,transform = train_transform )
full_test_dataset = ImageFolder(root = test_dir,transform = eval_transforms)
full_val_dataset = ImageFolder(root = val_dir,transform = eval_transforms)
train_loader = DataLoader(full_train_dataset, batch_size=batch_size, shuffle=True, num_workers=2,drop_last=False)
val_loader = DataLoader(full_val_dataset, batch_size=batch_size, shuffle=False, num_workers=2,drop_last=False)
test_loader = DataLoader(full_test_dataset, batch_size=batch_size, shuffle=False, num_workers=2,drop_last = False)


**Setup training components**

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters,lr = 0.001)
schdeuler = optim.lr_scheduler.ReduceLROnPlateau(
    optimizer,
    mode = 'max',
    factor = 0.5,
    patience = 3,
    verbose = True,
    min_lr = 1e-6
)


**TRAINING LOOP**
# i need to find a way to execute this on colab